In [1]:
#importing libraries
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import nltk
import re
import string
from nltk.stem import WordNetLemmatizer

In [2]:
import warnings


In [110]:
!pip install scipy
import scipy.sparse as sp

In [3]:
def remove_text_duplicates(df):
    df = df.drop_duplicates(subset='text', keep="first") #rimuovi i testi duplicati
    df = df[df['text'].notna()] #remove NaNs
    df = df.reset_index(drop=True) #reset index
    return df


In [4]:
def replace_char_df(df, char, subst_char, column_name='text'):
    warnings.filterwarnings('ignore')
    for i in range(len(df)):
        text=str(df[column_name][i])
        text=text.replace(char, subst_char)
        df[column_name][i]=text
    return df

In [5]:
def preproc(texts, lemmatizer, stopwords):
    prepr_texts=[]
    for i in range(len(texts)):
        review = re.sub('[^a-zA-Z]', ' ', texts[i])
        review = review.lower()
        review = review.split()
        review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
        review = ' '.join(review)
        prepr_texts.append(review)
    return prepr_texts


In [6]:
#reading the data
traintest = pd.read_pickle('C:/Users/dilan/Downloads/5. Analisi Statistica dei Dati/0. Tesi/final_aug.pkl') 
validation = pd.read_pickle('C:/Users/dilan/Downloads/5. Analisi Statistica dei Dati/0. Tesi/final_validation.pkl')

In [7]:
#stopword removal and lemmatization/stemming
from nltk.stem import SnowballStemmer


en_stopwords = nltk.corpus.stopwords.words('english')
it_stopwords = nltk.corpus.stopwords.words('italian')

en_lemmatizer = WordNetLemmatizer()
it_stemmer = SnowballStemmer('italian')

nltk.download('stopwords')

traintest.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dilan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,link,text,label,textlen,translated_text
0,https://www.liberoquotidiano.it/news/politica/...,05 maggio 2024 a\n\na\n\na\n\nForza Italia cre...,0,263,"May 5, 2024 a\n\na\n\na\n\nForza Italia is gro..."
1,https://www.liberoquotidiano.it/news/italia/39...,05 maggio 2024 a\n\na\n\na\n\nTragedia in Valt...,0,190,"May 5, 2024 a\n\na\n\na\n\nTragedy in Valtelli..."
2,https://www.liberoquotidiano.it/news/sport/392...,"05 maggio 2024 a\n\na\n\na\n\nScarico, demotiv...",0,353,"May 5, 2024 a\n\na\n\na\n\nExhausted, demotiva..."
3,https://www.liberoquotidiano.it/news/sport/392...,05 maggio 2024 a\n\na\n\na\n\nDomenica importa...,0,283,"May 5, 2024 a\n\na\n\na\n\nAn important Sunday..."
4,https://salerno.corriere.it/notizie/cronaca/24...,di Luigi Martino\n\nSolo sei iscritti tra i 3 ...,0,327,by Luigi Martino\n\nOnly six children between ...


In [8]:
traintest=replace_char_df(traintest, '[', '')
traintest=replace_char_df(traintest, '[', '', column_name='translated_text')
validation=replace_char_df(validation, '[', '')
validation=replace_char_df(validation, '[', '', column_name='translated_text')


In [9]:
traintest['tag']='tt'
validation['tag']='val'

In [14]:
df = pd.concat([traintest, validation], ignore_index=True)

In [16]:
texts=df['translated_text']

In [18]:
y=df['label']

In [20]:
tags=df['tag']

In [17]:
prepr_texts=preproc(texts, en_lemmatizer, en_stopwords)

In [21]:
# TOGLIERE DOPPIONI
millesimodataframe = {'text': prepr_texts, 'label': y, 'tag':tags
                     }

millesimodataframe = pd.DataFrame(millesimodataframe)
print(len(millesimodataframe))


nd_f_ing=remove_text_duplicates(millesimodataframe)
print(len(nd_f_ing))


1722
1408


In [98]:
nd_f_ing_x=nd_f_ing['text']

# tf idf INGLESE
tf_idf = TfidfVectorizer()
#applying tf idf to training data
tfidf = tf_idf.fit_transform(nd_f_ing_x)
#applying tf idf to training data
tfidf_ttval = tf_idf.transform(nd_f_ing_x)

print("n_samples: %d, n_features: %d" % tfidf_ttval.shape)

n_samples: 1408, n_features: 16534


In [99]:
nd_f_ing['label'].value_counts()

label
0    806
1    602
Name: count, dtype: int64

In [100]:
nd_y=nd_f_ing['label']
nd_tags=nd_f_ing['tag']

In [236]:
#CREA DF DI NUOVO
tfidf_df = {'text':nd_f_ing_x, 'tfidf': tfidf_ttval, 'label': nd_y, 'tag':nd_tags
                     }

tfidf_df = pd.DataFrame(tfidf_df)
print(len(tfidf_df))

1408


In [237]:
#DIVIDI IN DUE DF
grouped=tfidf_df.groupby(tfidf_df.tag)
traintest=grouped.get_group("tt")
validation=grouped.get_group("val")


In [238]:
traintest

,text,tfidf,label,tag
0,may forza italia growing poll proving sexy ant...,"(0, 16425)\t0.03502392823437498\n (0, 16397...",0,tt
1,may tragedy valtellina woman fell zipline died...,"(0, 16521)\t0.29042628127321013\n (0, 16460...",0,tt
2,may exhausted demotivated stunned crazy rumor ...,"(0, 16425)\t0.02707553427061538\n (0, 16347...",0,tt
3,may important sunday term salvation draw lecce...,"(0, 16462)\t0.08809776813024404\n (0, 16347...",0,tt
4,luigi martino six child year old registered mu...,"(0, 16460)\t0.09065708976460579\n (0, 16446...",0,tt
...,...,...,...,...
872,fifth paragraph constitution prescribes state ...,"(0, 16466)\t0.04476784189339855\n (0, 16440...",1,tt
873,fifth paragraph constitution prescribes state ...,"(0, 16466)\t0.04453005132174843\n (0, 16440...",1,tt
874,fifth paragraph constitution prescribes state ...,"(0, 16466)\t0.04476784189339855\n (0, 16440...",1,tt
875,th paragraph constitution prescribes state exa...,"(0, 16466)\t0.044750987365828644\n (0, 1644...",1,tt


In [104]:
ttx=traintest['tfidf']

In [105]:
tty=traintest['label']

In [106]:
from sklearn.model_selection import train_test_split
train_ratio = 0.80
test_ratio = 0.20
feature, feature_test, labels, labels_test = train_test_split(ttx, tty, test_size=1 - train_ratio)

# feature=feature.reset_index(drop=True)

# feature_test=feature_test.reset_index(drop=True)

# labels=labels.reset_index(drop=True)

# labels_test=labels_test.reset_index(drop=True)


In [107]:
labels=labels.astype('int')


In [111]:
# cross validation score
models = [
    RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),
    LinearSVC(),
    MultinomialNB()
]

cross_value_scored = []
for model in models:
    model_name = model.__class__.__name__
   
    # Convert feature to a dense NumPy array if it's sparse
    if sp.issparse(feature):
        feature_dense = feature.toarray()
    else:
        feature_dense = feature  # If not sparse, use feature directly

    # Check if feature_dense is an object array and contains sparse matrices
    if feature_dense.dtype == 'object' and any(sp.issparse(element) for element in feature_dense):
        feature_dense = np.vstack([element.toarray() for element in feature_dense]) # stack the elements of arrays vertically
    else:
        # Convert to float if not an object array or if it doesn't contain sparse matrices
        feature_dense = feature_dense.astype(float)  
    
    labels = labels.astype(int)
    accuracies = cross_val_score(model, feature_dense, labels, scoring='accuracy', cv=5)
    for accuracy in accuracies:
        cross_value_scored.append((model_name, accuracy))

In [112]:
df_cv = pd.DataFrame(cross_value_scored, columns =['model_name', 'accuracy'])
acc = pd.concat([df_cv.groupby('model_name').accuracy.mean(),df_cv.groupby('model_name').accuracy.std()], axis= 1,ignore_index=True)
acc.columns = ['Mean Accuracy', 'Standard deviation']
acc

,Mean Accuracy,Standard deviation
model_name,,
LinearSVC,0.997143,0.003912
MultinomialNB,0.980010,0.009331
RandomForestClassifier,0.880172,0.015470


# USARE GRIDSEARCH e cross validation 

In [114]:
rfc=RandomForestClassifier(random_state=42)
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'log2'],
    'max_depth' : [4,5,6,7,8,9,10],
    'criterion' :['gini', 'entropy']
}

kfold5 = KFold(shuffle = True, n_splits = 5)

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= kfold5, verbose=1, n_jobs=-1)
CV_rfc.fit(feature_dense, labels)


# rfc_rs = RandomizedSearchCV(rfc, 
#                          param_distributions = param_grid,
#                          cv = kfold_5,  
#                          n_iter = 5, 
#                          scoring = 'roc_auc', 
#                          verbose = 3, 
#                          n_jobs = -1)

Fitting 5 folds for each of 56 candidates, totalling 280 fits


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
             estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 5, 6, 7, 8, 9, 10],
                         'max_features': ['auto', 'log2'],
                         'n_estimators': [200, 500]},
             verbose=1)

In [115]:
# Convert feature to a dense NumPy array if it's sparse
if sp.issparse(feature_test):
    feature_test_dense = feature_test.toarray()
else:
    feature_test_dense = feature_test  # If not sparse, use feature directly

# Check if feature_dense is an object array and contains sparse matrices
if feature_test_dense.dtype == 'object' and any(sp.issparse(element) for element in feature_test_dense):
    feature_test_dense = np.vstack([element.toarray() for element in feature_test_dense]) # stack the elements of arrays vertically
else:
    # Convert to float if not an object array or if it doesn't contain sparse matrices
    feature_test_dense = feature_test_dense.astype(float)  

In [116]:
pred = CV_rfc.predict(feature_dense)
pred_test = CV_rfc.predict(feature_test_dense)

In [119]:
labels_test=pd.to_numeric(labels_test)

## risultati test

In [120]:
print("Confusion matrix:")
print(metrics.confusion_matrix(labels_test, pred_test))

Confusion matrix:
[[99  1]
 [11 65]]


In [121]:
print(metrics.classification_report(labels_test, pred_test, target_names=['Antimeridionalista', 'Neutro']))

                    precision    recall  f1-score   support

Antimeridionalista       0.90      0.99      0.94       100
            Neutro       0.98      0.86      0.92        76

          accuracy                           0.93       176
         macro avg       0.94      0.92      0.93       176
      weighted avg       0.94      0.93      0.93       176



## risultati validation

In [125]:
feature_val=validation['tfidf']
if sp.issparse(feature_val):
    feature_val = feature_val.toarray()
else:
    feature_val = feature_val

if feature_val.dtype == 'object' and any(sp.issparse(element) for element in feature_val):
    feature_val = np.vstack([element.toarray() for element in feature_val])
else:
    feature_val = feature_val.astype(float)  

In [123]:
labels_val=pd.to_numeric(validation['label'])

In [126]:
pred_val = CV_rfc.predict(feature_val)

In [127]:
print("Confusion matrix:")
print(metrics.confusion_matrix(labels_val, pred_val))

Confusion matrix:
[[300   0]
 [231   0]]


In [128]:
print(metrics.classification_report(labels_val, pred_val, target_names=['Antimeridionalista', 'Neutro']))

                    precision    recall  f1-score   support

Antimeridionalista       0.56      1.00      0.72       300
            Neutro       0.00      0.00      0.00       231

          accuracy                           0.56       531
         macro avg       0.28      0.50      0.36       531
      weighted avg       0.32      0.56      0.41       531



# Altri Classifiers

In [155]:
lr=LogisticRegression(C=0.02, penalty='none')
lr=lr.fit(feature_dense,labels)
pred_val = lr.predict(feature_val)


In [156]:
print("Confusion matrix:")
print(metrics.confusion_matrix(labels_val, pred_val))

Confusion matrix:
[[256  44]
 [189  42]]


In [157]:
print(metrics.classification_report(labels_val, pred_val, target_names=['Antimeridionalista', 'Neutro']))

                    precision    recall  f1-score   support

Antimeridionalista       0.58      0.85      0.69       300
            Neutro       0.49      0.18      0.26       231

          accuracy                           0.56       531
         macro avg       0.53      0.52      0.48       531
      weighted avg       0.54      0.56      0.50       531



In [230]:
from sklearn.svm import SVC

SVC_Gaussian=SVC(C=0.2, kernel='poly', degree=15, class_weight='balanced', gamma='auto')

In [231]:
svc=SVC_Gaussian.fit(feature_dense,labels)
pred_val=svc.predict(feature_val)


In [232]:
print("Confusion matrix:")
print(metrics.confusion_matrix(labels_val, pred_val))

Confusion matrix:
[[  0 300]
 [  0 231]]


In [233]:
print(metrics.classification_report(labels_val, pred_val, target_names=['Antimeridionalista', 'Neutro']))

                    precision    recall  f1-score   support

Antimeridionalista       0.00      0.00      0.00       300
            Neutro       0.44      1.00      0.61       231

          accuracy                           0.44       531
         macro avg       0.22      0.50      0.30       531
      weighted avg       0.19      0.44      0.26       531



# COSTRUZIONE DATASET CON ALTRE FEATURES

#### aggiungiamo altre features ai dataset traintest e validation, che al momento hanno 'text', 'tfidf' e 'label'

### nltk
A questo punto abbiamo già fatto tokenization/lemmatization/stemming, 

# CODICE VECCHIO

In [38]:
from sklearn.model_selection import GridSearchCV


In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
# from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy import stats
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns